# Data Cleaning

In [12]:
import os
import pandas as pd
import numpy as np

## Boston

In [2]:
boston_files = os.listdir('./Boston_Results/Original/')
boston_files

['Boston_Results_2009.csv',
 'Boston_Results_2010.csv',
 'Boston_Results_2011.csv',
 'Boston_Results_2012.csv',
 'Boston_Results_2013.csv',
 'Boston_Results_2014.csv',
 'Boston_Results_2015.csv',
 'Boston_Results_2016.csv',
 'Boston_Results_2017.csv']

In [3]:
boston_09 = pd.read_csv('./Boston_Results/Original/Boston_Results_2009.csv')
boston_10 = pd.read_csv('./Boston_Results/Original/Boston_Results_2010.csv')
boston_11 = pd.read_csv('./Boston_Results/Original/Boston_Results_2011.csv')
boston_12 = pd.read_csv('./Boston_Results/Original/Boston_Results_2012.csv')
boston_13 = pd.read_csv('./Boston_Results/Original/Boston_Results_2013.csv')
boston_14 = pd.read_csv('./Boston_Results/Original/Boston_Results_2014.csv')
boston_15 = pd.read_csv('./Boston_Results/Original/Boston_Results_2015.csv')
boston_16 = pd.read_csv('./Boston_Results/Original/Boston_Results_2016.csv')
boston_17 = pd.read_csv('./Boston_Results/Original/Boston_Results_2017.csv')

# drop problematic row from 2016
boston_16 = boston_16.iloc[:-1]

# binarize gender
boston_09['male'] = np.where(boston_09['M/F'] == 'M', 1, 0)
boston_10['male'] = np.where(boston_10['M/F'] == 'M', 1, 0)
boston_11['male'] = np.where(boston_11['M/F'] == 'M', 1, 0)
boston_12['male'] = np.where(boston_12['M/F'] == 'M', 1, 0)
boston_13['male'] = np.where(boston_13['M/F'] == 'M', 1, 0)
boston_14['male'] = np.where(boston_14['M/F'] == 'M', 1, 0)
boston_15['male'] = np.where(boston_15['M/F'] == 'M', 1, 0)
boston_16['male'] = np.where(boston_16['M/F'] == 'M', 1, 0)
boston_17['male'] = np.where(boston_17['M/F'] == 'M', 1, 0)

# split Official Time into hours, minuts, seconds
boston_09['time_h'] = boston_09['Official Time'].str[0].astype(int)
boston_10['time_h'] = boston_10['Official Time'].str[0].astype(int)
boston_11['time_h'] = boston_11['Official Time'].str[0].astype(int)
boston_12['time_h'] = boston_12['Official Time'].str[0].astype(int)
boston_13['time_h'] = boston_13['Official Time'].str[0].astype(int)
boston_14['time_h'] = boston_14['Official Time'].str[0].astype(int)
boston_15['time_h'] = boston_15['Official Time'].str[0].astype(int)
boston_16['time_h'] = boston_16['Official Time'].str[0].astype(int)
boston_17['time_h'] = boston_17['Official Time'].str[0].astype(int)

boston_09['time_m'] = boston_09['Official Time'].str[2:4].astype(int)
boston_10['time_m'] = boston_10['Official Time'].str[2:4].astype(int)
boston_11['time_m'] = boston_11['Official Time'].str[2:4].astype(int)
boston_12['time_m'] = boston_12['Official Time'].str[2:4].astype(int)
boston_13['time_m'] = boston_13['Official Time'].str[2:4].astype(int)
boston_14['time_m'] = boston_14['Official Time'].str[2:4].astype(int)
boston_15['time_m'] = boston_15['Official Time'].str[2:4].astype(int)
boston_16['time_m'] = boston_16['Official Time'].str[2:4].astype(int)
boston_17['time_m'] = boston_17['Official Time'].str[2:4].astype(int)

boston_09['time_s'] = boston_09['Official Time'].str[5:].astype(int)
boston_10['time_s'] = boston_10['Official Time'].str[5:].astype(int)
boston_11['time_s'] = boston_11['Official Time'].str[5:].astype(int)
boston_12['time_s'] = boston_12['Official Time'].str[5:].astype(int)
boston_13['time_s'] = boston_13['Official Time'].str[5:].astype(int)
boston_14['time_s'] = boston_14['Official Time'].str[5:].astype(int)
boston_15['time_s'] = boston_15['Official Time'].str[5:].astype(int)
boston_16['time_s'] = boston_16['Official Time'].str[5:].astype(int)
boston_17['time_s'] = boston_17['Official Time'].str[5:].astype(int)

# combine split times into total seconds
boston_09['time_seconds'] = boston_09['time_s'] + (boston_09['time_m'] * 60) + (boston_09['time_h'] * 60 * 60)
boston_10['time_seconds'] = boston_10['time_s'] + (boston_10['time_m'] * 60) + (boston_10['time_h'] * 60 * 60)
boston_11['time_seconds'] = boston_11['time_s'] + (boston_11['time_m'] * 60) + (boston_11['time_h'] * 60 * 60)
boston_12['time_seconds'] = boston_12['time_s'] + (boston_12['time_m'] * 60) + (boston_12['time_h'] * 60 * 60)
boston_13['time_seconds'] = boston_13['time_s'] + (boston_13['time_m'] * 60) + (boston_13['time_h'] * 60 * 60)
boston_14['time_seconds'] = boston_14['time_s'] + (boston_14['time_m'] * 60) + (boston_14['time_h'] * 60 * 60)
boston_15['time_seconds'] = boston_15['time_s'] + (boston_15['time_m'] * 60) + (boston_15['time_h'] * 60 * 60)
boston_16['time_seconds'] = boston_16['time_s'] + (boston_16['time_m'] * 60) + (boston_16['time_h'] * 60 * 60)
boston_17['time_seconds'] = boston_17['time_s'] + (boston_17['time_m'] * 60) + (boston_17['time_h'] * 60 * 60)

# drop unnecessary columns
drop_cols15 = ['Unnamed: 0', 'Bib', 'Name', 'M/F', 'City', 'State', 'Country',
       'Citizen', 'Unnamed: 9', '5K', '10K', '15K', '20K', 'Half', '25K',
       '30K', '35K', '40K', 'Pace', 'Proj Time', 'Official Time', 'Overall',
       'Gender', 'Division', 'time_h', 'time_m', 'time_s']

drop_cols = ['Unnamed: 8', 'Bib', 'Name', 'M/F', 'City', 'State', 'Country',
       'Citizen', '5K', '10K', '15K', '20K', 'Half', '25K',
       '30K', '35K', '40K', 'Pace', 'Proj Time', 'Official Time', 'Overall',
       'Gender', 'Division', 'time_h', 'time_m', 'time_s']

boston_09_clean = boston_09.drop(columns=drop_cols)
boston_10_clean = boston_10.drop(columns=drop_cols)
boston_11_clean = boston_11.drop(columns=drop_cols)
boston_12_clean = boston_12.drop(columns=drop_cols)
boston_13_clean = boston_13.drop(columns=drop_cols)
boston_14_clean = boston_14.drop(columns=drop_cols)
boston_15_clean = boston_15.drop(columns=drop_cols15)
boston_16_clean = boston_16.drop(columns=drop_cols)
boston_17_clean = boston_17.drop(columns=drop_cols)

# adjust column names
boston_09_clean.columns = ['age', 'male', 'time_seconds']
boston_10_clean.columns = ['age', 'male', 'time_seconds']
boston_11_clean.columns = ['age', 'male', 'time_seconds']
boston_12_clean.columns = ['age', 'male', 'time_seconds']
boston_13_clean.columns = ['age', 'male', 'time_seconds']
boston_14_clean.columns = ['age', 'male', 'time_seconds']
boston_15_clean.columns = ['age', 'male', 'time_seconds']
boston_16_clean.columns = ['age', 'male', 'time_seconds']
boston_17_clean.columns = ['age', 'male', 'time_seconds']


# add year column
boston_09_clean['year'] = 2009
boston_10_clean['year'] = 2010
boston_11_clean['year'] = 2011
boston_12_clean['year'] = 2012
boston_13_clean['year'] = 2013
boston_14_clean['year'] = 2014
boston_15_clean['year'] = 2015
boston_16_clean['year'] = 2016
boston_17_clean['year'] = 2017


# combine
boston_combined = pd.concat([boston_09_clean, boston_10_clean, boston_11_clean, 
                             boston_12_clean, boston_13_clean, boston_14_clean, 
                             boston_15_clean, boston_16_clean, boston_17_clean], ignore_index=True)

# reduce size
boston_combined['year'] = pd.to_numeric(boston_combined['year'], downcast='unsigned')
boston_combined['time_seconds'] = pd.to_numeric(boston_combined['time_seconds'], downcast='unsigned')
boston_combined['male'] = pd.to_numeric(boston_combined['male'], downcast='unsigned')
boston_combined['age'] = pd.to_numeric(boston_combined['age'], downcast='unsigned')



# save to csv
boston_09_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2009.csv', index=False)
boston_10_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2010.csv', index=False)
boston_11_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2011.csv', index=False)
boston_12_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2012.csv', index=False)
boston_13_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2013.csv', index=False)
boston_14_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2014.csv', index=False)
boston_15_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2015.csv', index=False)
boston_16_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2016.csv', index=False)
boston_17_clean.to_csv('./Boston_Results/Clean/Clean_Boston_Results_2017.csv', index=False)

boston_combined.to_csv('./Boston_Results/Clean/Clean_Boston_Results_Combined.csv', index=False)

In [4]:
boston_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190437 entries, 0 to 190436
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   age           190437 non-null  uint8 
 1   male          190437 non-null  uint8 
 2   time_seconds  190437 non-null  uint16
 3   year          190437 non-null  uint16
dtypes: uint16(2), uint8(2)
memory usage: 1.1 MB


## NYC

In [13]:
nyc_files = os.listdir('./NYC_Results/Original/')
nyc_files

['NYC_Results_2014.csv',
 'NYC_Results_2015.csv',
 'NYC_Results_2016.csv',
 'NYC_Results_2017.csv',
 'NYC_Results_2018.csv',
 'NYC_Results_2021.csv']

In [14]:
# read in files
nyc_14 = pd.read_csv('./NYC_Results/Original/NYC_Results_2014.csv')
nyc_15 = pd.read_csv('./NYC_Results/Original/NYC_Results_2015.csv')
nyc_16 = pd.read_csv('./NYC_Results/Original/NYC_Results_2016.csv')
nyc_17 = pd.read_csv('./NYC_Results/Original/NYC_Results_2017.csv')
nyc_18 = pd.read_csv('./NYC_Results/Original/NYC_Results_2018.csv')
nyc_21 = pd.read_csv('./NYC_Results/Original/NYC_Results_2021.csv')

# split groups into age and binarize gender
nyc_14['age'] = nyc_14['groups'].str[1:].astype(int)
nyc_15['age'] = nyc_15['groups'].str[1:].astype(int)
nyc_16['age'] = nyc_16['groups'].str[1:].astype(int)
nyc_17['age'] = nyc_17['groups'].str[1:].astype(int)
nyc_18['age'] = nyc_18['groups'].str[1:].astype(int)
nyc_21['age'] = nyc_21['groups'].str[1:].astype(int)

nyc_14['male'] = np.where(nyc_14['groups'].str[0] == 'M', 1, 0)
nyc_15['male'] = np.where(nyc_15['groups'].str[0] == 'M', 1, 0)
nyc_16['male'] = np.where(nyc_16['groups'].str[0] == 'M', 1, 0)
nyc_17['male'] = np.where(nyc_17['groups'].str[0] == 'M', 1, 0)
nyc_18['male'] = np.where(nyc_18['groups'].str[0] == 'M', 1, 0)
nyc_21['male'] = np.where(nyc_21['groups'].str[0] == 'M', 1, 0)

# split time into hours, minutes, seconds
nyc_14['time_h'] = nyc_14['times'].str[0].astype(int)
nyc_15['time_h'] = nyc_15['times'].str[0].astype(int)
nyc_16['time_h'] = nyc_16['times'].str[0].astype(int)
nyc_17['time_h'] = nyc_17['times'].str[0].astype(int)
nyc_18['time_h'] = nyc_18['times'].str[0].astype(int)
nyc_21['time_h'] = nyc_21['times'].str[0].astype(int)

nyc_14['time_m'] = nyc_14['times'].str[2:4].astype(int)
nyc_15['time_m'] = nyc_15['times'].str[2:4].astype(int)
nyc_16['time_m'] = nyc_16['times'].str[2:4].astype(int)
nyc_17['time_m'] = nyc_17['times'].str[2:4].astype(int)
nyc_18['time_m'] = nyc_18['times'].str[2:4].astype(int)
nyc_21['time_m'] = nyc_21['times'].str[2:4].astype(int)

nyc_14['time_s'] = nyc_14['times'].str[5:].astype(int)
nyc_15['time_s'] = nyc_15['times'].str[5:].astype(int)
nyc_16['time_s'] = nyc_16['times'].str[5:].astype(int)
nyc_17['time_s'] = nyc_17['times'].str[5:].astype(int)
nyc_18['time_s'] = nyc_18['times'].str[5:].astype(int)
nyc_21['time_s'] = nyc_21['times'].str[5:].astype(int)

# combine split times into total seconds
nyc_14['time_seconds'] = nyc_14['time_s'] + (nyc_14['time_m'] * 60) + (nyc_14['time_h'] * 60 * 60)
nyc_15['time_seconds'] = nyc_15['time_s'] + (nyc_15['time_m'] * 60) + (nyc_15['time_h'] * 60 * 60)
nyc_16['time_seconds'] = nyc_16['time_s'] + (nyc_16['time_m'] * 60) + (nyc_16['time_h'] * 60 * 60)
nyc_17['time_seconds'] = nyc_17['time_s'] + (nyc_17['time_m'] * 60) + (nyc_17['time_h'] * 60 * 60)
nyc_18['time_seconds'] = nyc_18['time_s'] + (nyc_18['time_m'] * 60) + (nyc_18['time_h'] * 60 * 60)
nyc_21['time_seconds'] = nyc_21['time_s'] + (nyc_21['time_m'] * 60) + (nyc_21['time_h'] * 60 * 60)

# drop unnecessary columns
nyc_14.drop(columns=['groups', 'times', 'time_h', 'time_m', 'time_s'], inplace=True)
nyc_15.drop(columns=['groups', 'times', 'time_h', 'time_m', 'time_s'], inplace=True)
nyc_16.drop(columns=['groups', 'times', 'time_h', 'time_m', 'time_s'], inplace=True)
nyc_17.drop(columns=['groups', 'times', 'time_h', 'time_m', 'time_s'], inplace=True)
nyc_18.drop(columns=['groups', 'times', 'time_h', 'time_m', 'time_s'], inplace=True)
nyc_21.drop(columns=['groups', 'times', 'time_h', 'time_m', 'time_s'], inplace=True)

# add year column
nyc_14['year'] = 2014
nyc_15['year'] = 2015
nyc_16['year'] = 2016
nyc_17['year'] = 2017
nyc_18['year'] = 2018
nyc_21['year'] = 2021

# combine
nyc_combined = pd.concat([nyc_14, nyc_15, nyc_16, nyc_17, nyc_18, nyc_21], ignore_index=True)

# reduce size
nyc_combined['year'] = pd.to_numeric(nyc_combined['year'], downcast='unsigned')
nyc_combined['time_seconds'] = pd.to_numeric(nyc_combined['time_seconds'], downcast='unsigned')
nyc_combined['male'] = pd.to_numeric(nyc_combined['male'], downcast='unsigned')
nyc_combined['age'] = pd.to_numeric(nyc_combined['age'], downcast='unsigned')


# save cleaned sets to csv
nyc_14.to_csv('./NYC_Results/Clean/Clean_NYC_Results_2014.csv', index=False)
nyc_15.to_csv('./NYC_Results/Clean/Clean_NYC_Results_2015.csv', index=False)
nyc_16.to_csv('./NYC_Results/Clean/Clean_NYC_Results_2016.csv', index=False)
nyc_17.to_csv('./NYC_Results/Clean/Clean_NYC_Results_2017.csv', index=False)
nyc_18.to_csv('./NYC_Results/Clean/Clean_NYC_Results_2018.csv', index=False)
nyc_21.to_csv('./NYC_Results/Clean/Clean_NYC_Results_2021.csv', index=False)

nyc_combined.to_csv('./NYC_Results/Clean/Clean_NYC_Results_Combined.csv', index=False)

In [15]:
nyc_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59975 entries, 0 to 59974
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   age           59975 non-null  uint8 
 1   male          59975 non-null  uint8 
 2   time_seconds  59975 non-null  uint16
 3   year          59975 non-null  uint16
dtypes: uint16(2), uint8(2)
memory usage: 351.5 KB


## Berlin

In [8]:
berlin_files = os.listdir('./Berlin_Results/Original/')
berlin_files

['Berlin_Marathon_data_1974_2019.csv',
 'Berlin_Marathon_data_1974_2019_alt.csv',
 'Berlin_Marathon_weather_data_since_1974.csv']

In [9]:
berlin = pd.read_csv('./Berlin_Results/Original/Berlin_Marathon_data_1974_2019_alt.csv')

# binarize gender
berlin['male'] = np.where(berlin['GENDER'] == 'male', 1, 0)


# drop rows with no time
berlin = pd.DataFrame(berlin[berlin['TIME'] != 'no time'])

# split time into hours, minutes, seconds
berlin['time_h'] = berlin['TIME'].str[0].astype(int)
berlin['time_m'] = berlin['TIME'].str[2:4].astype(int)
berlin['time_s'] = berlin['TIME'].str[5:].astype(int)


# combine split times into total seconds
berlin['time_seconds'] = berlin['time_s'] + (berlin['time_m'] * 60) + (berlin['time_h'] * 60 * 60)


# drop unnecessary columns
berlin.drop(columns=['COUNTRY', 'TIME', 'GENDER', 'time_h', 'time_m', 'time_s'], inplace=True)

# adjust column names
berlin.columns = ['year', 'age', 'male', 'time_seconds']

# reduce size
berlin['year'] = pd.to_numeric(berlin['year'], downcast='unsigned')
berlin['age'] = pd.to_numeric(berlin['age'], downcast='unsigned')
berlin['male'] = pd.to_numeric(berlin['male'], downcast='unsigned')
berlin['time_seconds'] = pd.to_numeric(berlin['time_seconds'], downcast='unsigned')

# save cleaned sets to csv
berlin.to_csv('./Berlin_Results/Clean/Clean_Berlin_Results_1974_2019.csv', index=False)

C:\Users\David\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
berlin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 869474 entries, 0 to 871182
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          869474 non-null  uint16
 1   age           869474 non-null  uint8 
 2   male          869474 non-null  uint8 
 3   time_seconds  869474 non-null  uint16
dtypes: uint16(2), uint8(2)
memory usage: 11.6 MB
